# 两相图像自动分割全流程演示

本 Notebook 演示如何使用 `phase_segmentation` 包，将原始灰度图像自动分割为两个相，并在每一步保持可视化。

> ⚠️ 请在运行前修改下方代码中的 `image_path` 为你本地的灰度图像路径。


In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
from PIL import Image, ImageFilter

from phase_segmentation import Segmenter
from phase_segmentation.segmentation import otsu_threshold, binary_closing


In [ ]:
# 指定要处理的灰度图像路径
image_path = Path("path/to/your/image.png")
assert image_path.exists(), f'找不到图像: {image_path}'

original = Image.open(image_path).convert('L')
plt.figure(figsize=(6, 6))
plt.imshow(original, cmap='gray')
plt.title('原始灰度图像')
plt.axis('off')
plt.show()


In [ ]:
# 1. 可选的高斯平滑，便于观察模糊效果
blur_radius = 1.5
blurred = original.filter(ImageFilter.GaussianBlur(radius=blur_radius))

plt.figure(figsize=(6, 6))
plt.imshow(blurred, cmap='gray')
plt.title(f'高斯平滑 (radius={blur_radius})')
plt.axis('off')
plt.show()

# 2. 计算 Otsu 阈值并得到二值掩膜
blurred_array = np.array(blurred, dtype=np.uint8)
threshold = otsu_threshold(blurred_array)
mask = (blurred_array <= threshold).astype(np.uint8)

plt.figure(figsize=(6, 6))
plt.imshow(mask, cmap='gray')
plt.title(f'Otsu 二值掩膜 (threshold={threshold:.2f})')
plt.axis('off')
plt.show()


In [ ]:
# 3. 形态学闭运算清理掩膜
morphology_radius = 2
clean_mask = binary_closing(mask.astype(bool), morphology_radius).astype(np.uint8)

plt.figure(figsize=(6, 6))
plt.imshow(clean_mask, cmap='gray')
plt.title(f'形态学闭运算后掩膜 (radius={morphology_radius})')
plt.axis('off')
plt.show()

# 4. 使用 Segmenter 直接得到完整结果，并叠加边界
segmenter = Segmenter(
    blur_radius=blur_radius,
    morphology_radius=morphology_radius,
    select_dark_phase=True,
    closing=True,
)
result = segmenter.segment(original)
overlay = result.overlay_on(original.convert('RGB'))

fig, axes = plt.subplots(1, 2, figsize=(12, 6))
axes[0].imshow(result.mask, cmap='gray')
axes[0].set_title('Segmenter 输出掩膜')
axes[0].axis('off')

axes[1].imshow(overlay)
axes[1].set_title('原图 + 边界覆盖')
axes[1].axis('off')

plt.tight_layout()
plt.show()
